_Autor:_    __Jesús Casado__ <br> _Revisión:_ __5/11/2019__ <br>

__Introducción__<br>
Funciones para extraer datos de las bases de datos Spain02: mapas diarios de precipitación y temperatura (máxima, mínima y media) de toda España a resolución aprox. 10 km.

__Cosas que arreglar__ <br>

***

__Índice__ <br>

In [5]:
import numpy as np
import pandas as pd
import datetime
import os
from netCDF4 import Dataset
from pyproj import Transformer
os.environ['PROJ_LIB'] = r'C:\Anaconda3\pkgs\proj4-4.9.3-vc14_5\Library\share'

In [2]:
def extract_Spain02(variable, path, lonlim=None, latlim=None, datelim=None, coordsIn='epsg:4326',
                    coordsOut=None, dateformat='%Y-%m-%d', plot=False, color='steelblue'):
    """Extrae los datos de Spain02 v5 para la variable y espacio definidos.
    
    Entradas:
    ---------
    variable:    string. Abreviatura de la variable en la base de datos: 'pr', precipitación; 'tas', temperatura...
    path:        string. Donde se encuentran los archivos .nc de Spain02
    lonlim:      list(2,1). Límites de longitud del rectángulo de búsqueda. Si es 'None' se extraen todas las longitudes disponibles
    latlim:      list(2,1). Límites de latitud del rectángulo de búsqueda. Si es 'None' se extraen todas las latitudes disponibles
    datelim:     list of strings (2,1). Fechas límites de la serie a extraer. Si es 'None' se extraen todas las fechas disponibles
    coordsIn:    string. Código EPSG del sistema de coords de 'lonlim' y 'latlim'. Por defecto WGS84 en coordenadas geográficas
    coordsOut:   string. Código EPSG del sistema de coords de los datos de salida. Por defecto (None) se toma el mismo que 'coordsIn'
    dateformat:  string. Formato de las fechas en 'datelim'
    plot:        boolena. Si se quiere plotear la serie
    color:       string. Color del gráfico
    
    Salidas:
    --------
    Como métodos de la función:
    data:        array 3D [dates, lat, lon]. Datos extraídos
    dates:       array 1D. Fechas en 'data'
    Si las coordenadas son geográficas:
    lat:         array 1D. Latitudes en 'data'
    lon:         array 1D. Longitudes en 'data'
    Si las coordenadas son proyectadas:
    X:           array 2D. Coordenada x de cada celda del mapa diario
    Y:           array 2D. Coordenada y de cada celda del mapa diario
    """
    
    # comprobaciones previas: 
    if (lonlim != None) & (latlim != None):
        if len(lonlim) != 2 or len(latlim) != 2:
            print('ERROR. Longitud errónea de "latlim" o "lonlim".')
            return
        if coordsIn != 'epsg:4326':
            toWGS84 = Transformer.from_crs(coordsIn, 'epsg:4326')
            latlim, lonlim = toWGS84.transform(lonlim, latlim)
    
    # cargar NetCDF
    nc = Dataset(path + 'Spain02_v5.0_DD_010reg_aa3d_' + variable + '.nc', 'r')
    data = nc[variable][::].copy()
    # convertir en 0 valores por debajo de la décima de mm
    if variable == 'pr':
        data[data < .1] = 0
    
    # variable tiempo (fechas)
    time = nc['time'][::].data
    dates = np.array([date.date() for date in pd.date_range('1950-1-1', '2015-12-31')])
    if datelim != None:
        if len(datelim) != 2:
            print('ERROR. Longitud errónea de "datelim".')
            return
        datelim = [datetime.datetime.strptime(lim, dateformat).date() for lim in datelim]
        maskdate = (dates >= datelim[0]) & (dates <= datelim[-1])
        extract_Spain02.dates = dates[maskdate]
        data = data[maskdate, :, :]
    else:
        extract_Spain02.dates = dates
    
    # variable latitud
    lat = nc['lat'][::].data
    if latlim != None:
        masklat = (lat >= latlim[0]) & (lat <= latlim[-1])
        data = data[:, masklat, :]
        lat = lat[masklat]
    
    # variable longitud
    lon = nc['lon'][::].data
    if lonlim != None:
        masklon = (lon >= lonlim[0]) & (lon <= lonlim[-1])
        data = data[:, :, masklon]
        lon = lon[masklon]
    
    # guardar 'array' con datos extraídos
    extract_Spain02.data = data
    
    # guardar latitud y longitud en el sistema 'coords'
    if coordsOut == None:
        coordsOut = coordsIn
    if coordsOut != 'epsg:4326':
        # matrices de longitud y latitud de cada una de las celdas
        lonaux, lataux = np.meshgrid(lon, lat)
        # transformar coordendas y reformar en matrices del mismo tamaño que el mapa diario
        fromWGS84 = Transformer.from_crs('epsg:4326', coordsOut)
        yaux, xaux= fromWGS84.transform(lonaux.flatten(), lataux.flatten())
        extract_Spain02.X = xaux.reshape(lonaux.shape)
        extract_Spain02.Y = yaux.reshape(lataux.shape)
    else:
        extract_Spain02.lat = lat
        extract_Spain02.lon = lon
        
    if plot == True:
        fig, ax = plt.subplots(figsize=(14, 3))
        for i in range(data.shape[1]):
            for j in range(data.shape[2]):
                ax.plot(dates, data[:, i, j], lw=.2, c=color, alpha=.1)
        ax.tick_params(labelsize=11)
        ax.set(xlim=(dates[0], dates[-1]))
        ylabel = {'pr': 'Pd (mm)', 'tas': 'Tmed (ºC)'}
        ax.set_ylabel(ylabel[variable], fontsize=13)
        ax.set_title('Spain02', fontsize=14, fontweight='bold');